<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#建立" data-toc-modified-id="建立-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>建立</a></span></li><li><span><a href="#预处理和建立模型" data-toc-modified-id="预处理和建立模型-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>预处理和建立模型</a></span></li><li><span><a href="#训练" data-toc-modified-id="训练-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>训练</a></span></li></ul></li></ul></div>

_字符型语言模型_

In [1]:
%matplotlib inline
import imp
import utils
from utils import *
from __future__ import division, print_function

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [1]:
from keras.layers import TimeDistributed,Activation
from numpy.random import choice

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 建立

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [7]:
path = get_file('nietzsche.txt',origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt") # 尼采的诗歌
text = open(path).read().lower()
print("语料库长度：",len(text))

语料库长度： 600893


In [101]:
#path = 'data/wiki/'
#text = open(path+'small.txt').read().lower()
#print('corpus length:', len(text))

#text = text[0:1000000]

corpus length: 137587200


In [8]:
# 字符型语言模型的字典,只包含不重复的字符，具体见吴恩达课程
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1
print('total chars:',vocab_size)

total chars: 59


In [9]:
chars.insert(0,"\0")

In [10]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxy'

In [11]:
char_indices = dict((c,i) for i,c in enumerate(chars)) # 字符到索引

In [12]:
indices_char = dict((i, c) for i, c in enumerate(chars)) # 索引 字符 字典

In [13]:
idx = [char_indices[c] for c in text] # 改预料转成数字索引数组

In [14]:
idx[:10]

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]

In [15]:
# 再把数字索引转成字符

''.join(indices_char[i] for i in idx[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

## 预处理和建立模型

In [16]:
maxLen = 40 # 最大长度
sentences = [] # 生成的句子
next_chars = [] # 下一个字符

In [17]:
len(idx)-maxLen+1

600854

In [18]:
for i in range(0,len(idx)-maxLen+1):
    sentences.append(idx[i:i+maxLen])
    next_chars.append(idx[i+1:i+maxLen+1]) # 下一个字符
print('nb sequences:', len(sentences))

nb sequences: 600854


In [19]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]]) # 拼接起来转化成np格式
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [20]:
sentences.shape,next_chars.shape

((600852, 40), (600852, 40))

In [21]:
n_fac = 24 # 嵌入矩阵输出大小

In [22]:
# return_sequences: 布尔值。是返回输出序列中的最后一个输出，还是全部序列
# dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于输入的线性转换。
# recurrent_dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于循环层状态的线性转换。
# implementation: 实现模式，1 或 2。 模式 1 将把它的操作结构化为更多的小的点积和加法操作， 
# 而模式 2 将把它们分批到更少，更大的操作中。 这些模式在不同的硬件和不同的应用中具有不同的性能配置文件。
model = Sequential([
    Embedding(input_dim=vocab_size,output_dim=n_fac,input_length=maxLen),
    LSTM(units=512,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,implementation=2,input_shape=[None,n_fac]),
    Dropout(0.2),
    LSTM(units=512,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,implementation=2),
    Dropout(0.2),
    TimeDistributed(Dense(vocab_size)),
    Activation('softmax')
])

model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam())

In [50]:
# model=Sequential([
#         Embedding(vocab_size, n_fac, input_length=maxLen),
#         LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
#              consume_less='gpu'),
#         Dropout(0.2),
#         LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
#              consume_less='gpu'),
#         Dropout(0.2),
#         TimeDistributed(Dense(vocab_size)),
#         Activation('softmax')
#     ])    

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, input_shape=(None, 24), dropout=0.2, recurrent_dropout=0.2, implementation=2)`
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, implementation=2)`
  import sys


## 训练

In [ ]:
np.expand_dims()

In [23]:
next_chars.shape

(600852, 40)

In [24]:
np.expand_dims(next_chars,-1).shape

(600852, 40, 1)

In [25]:
sentences.shape

(600852, 40)

In [27]:
# 没看懂
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [26]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64,epochs=1)

Epoch 1/1
600852/600852 [==============================] - 744s 1ms/step - loss: 1.5029


In [28]:
print_example()

ethics is a basic foundation of all that "oly conduct," as moral unaqual receives. hence this tyranny of critics. the hororical consciousness
of
the exercise of human cruelty.

146. all the sun of religion, licated wagner, and i say.

269. we have come whatever decorations of cimilarizations and
hus birdold himself, which have hornor with the method of arist


In [29]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64,epochs=1)

Epoch 1/1
600852/600852 [==============================] - 731s 1ms/step - loss: 1.2801


In [222]:
print_example()

ethics is a basic foundation of all that he maluces indofely and is; pticrast', and re onerog je ivesantamale as whered
and ror and kytinf? on chaninn nurdeln--ans prory. heke the pepadinar; anf bom,
puntely"" ones to bucf, alcherstol the qisleves: the the wite dadong the gur is prang not galcaula rewinl
more by than sic appads not pepow o mee, a more
bins c


In [235]:
model.optimizer.lr=0.001

In [236]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 640s - loss: 1.5152   


In [30]:
print_example()

ethics is a basic foundation of all that men but now, that men to be contempt and relative and involved. hence perhaps impart a consequences
with all sentiments,
or did be that he is closely the reason greater,
men, however, who
are inverted even more anything else, are found, creatness, perhaps a man, and if he saw made in the same advantage of past. to who


In [31]:
model.optimizer.lr=0.0001

In [33]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1)

Epoch 1/1
600852/600852 [==============================] - 736s 1ms/step - loss: 1.2452


In [34]:
print_example()

ethics is a basic foundation of all that
truths because of a repulsive men has fixed men that he rather right:
and if the whole revil which we just as one should certainly the short of knowledge remains: but there is almost after the intellectual
pure impulses of view of
their own, a
jhilosopher
clishes, it is not perhaps a means of virtue. on the whole reti


In [35]:
model.save_weights('data/char_rnn.h5')

In [36]:
model.optimizer.lr=0.00001

In [38]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64,epochs=1)

Epoch 1/1
600852/600852 [==============================] - 728s 1ms/step - loss: 1.2105


In [39]:
print_example()

ethics is a basic foundation of all that was obliged to
stand by them accompanying subjection.

146. supposing that there is above all something of the moral forms and causes, cannot now wink and concealing
them. through anything that type of love, of
truth for the way, by false and absolute state of friendliness and looks, very
forgotten, and even of there 


In [40]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, epochs=1)

Epoch 1/1
600852/600852 [==============================] - 737s 1ms/step - loss: 1.1975


In [41]:
print_example()

ethics is a basic foundation of all that you had to
contradict the intentions, or at least _come to each other." kour faith for personal man is comprehensible for an
animal, and make
them to have understand something boundlessly
egoistic"? around a terror, when a cogitation which 1ow refreshly--and how much as much
of concealment! it to deceive whom circumst


In [42]:
print_example()

ethics is a basic foundation of all that is obvious
and belittle in all truths.


27

=every older person is would make it, in which are something
else is willingly compromised and not by such judgment, the
fellow-men have arisen.


51

=ghort, supposing that it is his enemy is
of--every one who has false works upon the south which is the survival of
art and


In [43]:
model.save_weights('data/char_rnn.h5')